<a href="https://colab.research.google.com/github/ryann-arruda/deep_learning_algorithms/blob/main/perceptron/pytorch_implementation/multilayer_perceptron/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F

## **Loading Dataset**

In [25]:
data = load_iris()

In [26]:
x = data.data

In [27]:
y = data.target

## **Data Normalization**

In [28]:
x = (x - x.min(axis=0))/(x.max(axis=0) - x.min(axis=0))

## **Separating the Dataset**

In [29]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=27, shuffle=True, stratify=y)

In [30]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=27)

## **Creating the Dataset Object**

In [31]:
class SpecificDataset(Dataset):

  def __init__(self, x, y):
    self.features = torch.tensor(x, dtype=torch.float32)
    self.labels = torch.tensor(y, dtype=torch.float32)

  def __getitem__(self, index):
    x = self.features[index]
    y = self.labels[index]

    return x, y

  def __len__(self):
    return self.labels.shape[0]

In [32]:
train_dataset = SpecificDataset(x_train, y_train)

In [33]:
test_dataset = SpecificDataset(x_test, y_test)

In [34]:
val_dataset = SpecificDataset(x_val, y_val)

In [35]:
train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=10,
                              shuffle=True)

In [36]:
test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=10,
                             shuffle=False)

In [37]:
val_dataloader = DataLoader(dataset=val_dataset,
                            batch_size=10,
                            shuffle=False)

## **MLP**

In [38]:
class MLP(torch.nn.Module):

  def __init__(self, input, output):
    super().__init__()

    self.layers = torch.nn.Sequential(
        torch.nn.Linear(input, 8),
        torch.nn.ReLU(),

        torch.nn.Linear(8, 8),
        torch.nn.ReLU(),

        torch.nn.Linear(8, output),
    )

  def forward(self, x):
    return self.layers(x)

In [39]:
torch.manual_seed(27)

In [58]:
model = MLP(4, 4)

In [59]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.3)

## **Training**

In [60]:
def accuracy(model, dataloader):
  accuracy = 0.0
  total = 0

  model.eval()

  for index, (x, y) in enumerate(dataloader):

    with torch.inference_mode():
      z = model(x)

    labels = torch.argmax(z, dim=1)
    labels = labels.view(y.shape).to(y.dtype)

    results = labels == y
    accuracy += results.sum().item()
    total += len(results)

  return accuracy / total

In [61]:
epochs = 10

In [62]:
for epoch in range(epochs):
  model.train()

  for index, (x, y) in enumerate(train_dataloader):
    z = model(x)
    y = y.long()
    loss = F.cross_entropy(z, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch: {epoch + 1:03d}/{epochs:03d} | Batch: {index + 1:03d}/{len(train_dataloader):03d} | Loss: {loss:.3f}')

  print(f'Train Accuracy: {accuracy(model, train_dataloader)*100: .2f} | Val Accuracy: {accuracy(model, val_dataloader)*100: .2f}%')

Epoch: 001/010 | Batch: 001/011 | Loss: 1.354
Epoch: 001/010 | Batch: 002/011 | Loss: 1.354
Epoch: 001/010 | Batch: 003/011 | Loss: 1.300
Epoch: 001/010 | Batch: 004/011 | Loss: 1.285
Epoch: 001/010 | Batch: 005/011 | Loss: 1.251
Epoch: 001/010 | Batch: 006/011 | Loss: 1.252
Epoch: 001/010 | Batch: 007/011 | Loss: 1.241
Epoch: 001/010 | Batch: 008/011 | Loss: 1.218
Epoch: 001/010 | Batch: 009/011 | Loss: 1.240
Epoch: 001/010 | Batch: 010/011 | Loss: 1.216
Epoch: 001/010 | Batch: 011/011 | Loss: 1.170
Train Accuracy:  35.19 | Val Accuracy:  16.67%
Epoch: 002/010 | Batch: 001/011 | Loss: 1.196
Epoch: 002/010 | Batch: 002/011 | Loss: 1.187
Epoch: 002/010 | Batch: 003/011 | Loss: 1.135
Epoch: 002/010 | Batch: 004/011 | Loss: 1.178
Epoch: 002/010 | Batch: 005/011 | Loss: 1.244
Epoch: 002/010 | Batch: 006/011 | Loss: 1.144
Epoch: 002/010 | Batch: 007/011 | Loss: 1.131
Epoch: 002/010 | Batch: 008/011 | Loss: 1.144
Epoch: 002/010 | Batch: 009/011 | Loss: 1.114
Epoch: 002/010 | Batch: 010/011 |

## **Evaluating the Results**

In [63]:
print(f'Accuracy (test): {accuracy(model, test_dataloader):.2f}')

Accuracy (test): 0.97
